In [5]:
import folium
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd

In [13]:
# Load the data into pandas

dataframe_ksi = pd.read_csv('./data/KSI.csv')
dataframe_ksi.head()

,X,Y,INDEX_,ACCNUM,YEAR,DATE,TIME,STREET1,STREET2,OFFSET,...,AG_DRIV,REDLIGHT,ALCOHOL,DISABILITY,HOOD_158,NEIGHBOURHOOD_158,HOOD_140,NEIGHBOURHOOD_140,DIVISION,ObjectId
0,-8.844611e+06,5.412414e+06,3387730,892658.0,2006,2006/03/11 05:00:00+00,852,BLOOR ST W,DUNDAS ST W,NaN,...,Yes,NaN,NaN,NaN,88,High Park North,88,High Park North (88),D11,1
1,-8.844611e+06,5.412414e+06,3387731,892658.0,2006,2006/03/11 05:00:00+00,852,BLOOR ST W,DUNDAS ST W,NaN,...,Yes,NaN,NaN,NaN,88,High Park North,88,High Park North (88),D11,2
2,-8.816480e+06,5.434843e+06,3388101,892810.0,2006,2006/03/11 05:00:00+00,915,MORNINGSIDE AVE,SHEPPARD AVE E,NaN,...,Yes,Yes,NaN,NaN,146,Malvern East,132,Malvern (132),D42,3
3,-8.829728e+06,5.419071e+06,3389067,893184.0,2006,2006/01/01 05:00:00+00,236,WOODBINE AVE,O CONNOR DR,NaN,...,Yes,NaN,Yes,NaN,60,Woodbine-Lumsden,60,Woodbine-Lumsden (60),D55,4
4,-8.816480e+06,5.434843e+06,3388102,892810.0,2006,2006/03/11 05:00:00+00,915,MORNINGSIDE AVE,SHEPPARD AVE E,NaN,...,Yes,Yes,NaN,NaN,146,Malvern East,132,Malvern (132),D42,5


In [22]:
print(dataframe_ksi['AG_DRIV'].unique())
print(dataframe_ksi['REDLIGHT'].unique())
print(dataframe_ksi['ALCOHOL'].unique())
print(dataframe_ksi['DISABILITY'].unique())

['Yes' nan]
[nan 'Yes']
[nan 'Yes']
[nan 'Yes']


In [24]:
ksi_df = dataframe_ksi

ag_driv_count = ksi_df['AG_DRIV'].value_counts()
redlight_count = ksi_df['REDLIGHT'].value_counts()
alcohol_count = ksi_df['ALCOHOL'].value_counts()
disability_count = ksi_df['DISABILITY'].value_counts()

print(ag_driv_count)
print(redlight_count)
print(alcohol_count)
print(disability_count)


AG_DRIV
Yes    9460
Name: count, dtype: int64
REDLIGHT
Yes    1520
Name: count, dtype: int64
ALCOHOL
Yes    788
Name: count, dtype: int64
DISABILITY
Yes    486
Name: count, dtype: int64


In [14]:
# Create a map of Toronto with the KSI data
shapefile_path = './data/KSI.shp'
ksi_shp = gpd.read_file(shapefile_path)

In [8]:
ksi_shp.crs

<Projected CRS: EPSG:3857>
Name: WGS 84 / Pseudo-Mercator
Axis Info [cartesian]:
- X[east]: Easting (metre)
- Y[north]: Northing (metre)
Area of Use:
- name: World between 85.06°S and 85.06°N.
- bounds: (-180.0, -85.06, 180.0, 85.06)
Coordinate Operation:
- name: Popular Visualisation Pseudo-Mercator
- method: Popular Visualisation Pseudo Mercator
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [16]:
ksi_shp_wgs84 = ksi_shp.to_crs(epsg=4326)
ksi_coordinates = ksi_shp_wgs84['geometry'].apply(lambda geom: (geom.y, geom.x)).tolist()
toronto_center = [43.651070, -79.347015] 



In [27]:
folium_map_toronto = folium.Map(location=toronto_center, zoom_start=12)
from folium.plugins import HeatMap
from folium.map import LayerControl


In [28]:
def create_heatmap(dataframe, name):
    heatmap_data = dataframe['geometry'].apply(
        lambda geom: (geom.y, geom.x)).tolist()
    return HeatMap(heatmap_data, name=name, show=False)


heatmap_layers = {
    'Aggressive Driving': create_heatmap(ksi_shp_wgs84[ksi_shp_wgs84['AG_DRIV'] == 'Yes'], 'Aggressive Driving'),
    'Redlight': create_heatmap(ksi_shp_wgs84[ksi_shp_wgs84['REDLIGHT'] == 'Yes'], 'Redlight'),
    'Alcohol Related': create_heatmap(ksi_shp_wgs84[ksi_shp_wgs84['ALCOHOL'] == 'Yes'], 'Alcohol Related'),
    'Disability Related': create_heatmap(ksi_shp_wgs84[ksi_shp_wgs84['DISABILITY'] == 'Yes'], 'Disability Related')
}

In [29]:
for layer_name, i in heatmap_layers.items():
    i.add_to(folium_map_toronto)

LayerControl().add_to(folium_map_toronto)

In [30]:
heatmap_html_file_path ='./'
folium_map_toronto.save(heatmap_html_file_path + 'KSI_heatmap.html')